# Calculate distance and age for G150 using HI velocities

In [33]:
%matplotlib inline
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from pyJam.fermi.SNR_MC.distNsep import *
import pyJam.kdistMaster as kd
from astropy import units as u
from astropy import constants as c
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
mpl.style.use('ggplot')
from pyJam.utilities.sortNumStrings import atoi, natural_keys

%load_ext autoreload
%autoreload 2

home = os.environ['HOME']
desk = os.path.join(home,'Desktop')
#os.chdir(os.path.join(home,'Dropbox/CurrentWork/G150/'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## HI, radio, and other properies

In [6]:
#For Galactic rotation curve, only negative velocities are allowed (why?), so left out 2.9 km/s
#Vlsr HI velocties[km/s], all with widths 5 km/s or less, so there is no evidence of line broadening from the SNR shock
HIvels = [-44.7, -35.9, -6.9]# left out 2.9] 

#Radio coords of the SNR
lrad = 150.3
brad = 4.5

#For rotation curve
#kidst defaults are r0=8.4e3,v0=2.54e2 from Reid et al., 2009
#Reid et al 2014 gives r0=8.34, v0=240
R0 = 8.34  # [kpc] distance to center of Galaxy
V0 = 240 # [km/s] LSR velocity 

# 1 GeV-1 TeV fit disk propertie
R_gev = 1.40 # My best fit > 1 GeV #1.46 was from ajay's poster #pointlike uniform disk fit radius [deg], +/- 0.03
lgev = 150.35
bgev = 4.551
#Distance Jack listed on the symposium poster
jacksDists= [5.5,4,0.6] #kpc
jacksSizes = np.array([288, 209,31]) #pc

## Use the fit GeV coordinates

In [34]:
dist = np.vectorize(kd.kdist) #in kpc
dnear = dist(l = lgev, b = bgev, vin = HIvels, near = True,r0 = R0,v0=V0)
dfar= dist(l = lgev, b = bgev, vin = HIvels, near = False,r0 = R0,v0=V0)
print 'HI velocites are %s km/s' %HIvels
print 'Near distances %s kpc' %dnear
print 'Far distances %s kpc \n' %dfar


HI velocites are [-44.7, -35.9, -6.9] km/s
Near distances [ 5.55732525  3.94174137  0.38461266] kpc
Far distances [ 5.55732525  3.94174137  0.38461266] kpc 



## Projected diameter
### My numbers are a little different than Jack's. I don't think mine are wrong. If I do it in astropy I get my results

In [35]:
size = np.vectorize(physSep) #diam
sizes = size(t=R_gev*2, d=dnear) #t is for theta = angular radius
rads = sizes/2.
jsizes = size(t=R_gev*2, d=jacksDists) #t is for theta = angular radius, 2* for diameter/size
jrads = jsizes/2
print 'Physical of source in pc: %s' %sizes
print "Physical of source (using Jack's dists) in pc : %s" %jsizes
print "Physical sizes from fermi symp poster in pc: %s" %jacksSizes

Physical of source in pc: [ 271.58214487  192.62982233   18.79572025]
Physical of source (using Jack's dists) in pc : [ 268.78070481  195.47687622   29.32153143]
Physical sizes from fermi symp poster in pc: [288 209  31]


In [9]:
dFromSizes(18.79572025,R_gev*2)

0.38461265829770591

## For SNRs in the LMC, SMC and M33 there is a sharp cutoff in the SNR size distribution above D~60pc, which implies a distance of 1.3 kpc. Use this below

## Our closest distances are pretty off and it will make a big difference in the age!

In [10]:
dmaxJ = 1.3 #kpc
smax = 60 #pc
maxSize = physSep(t=R_gev*2, d=1.3)
print 'Using 1 GeV fit radius and d = %s kpc, size = %s' %(dmaxJ,maxSize)

dmax = dFromSizes(smax,R_gev*2)
print 'Max distance assuming 60pc diameter and > 1 GeV fit radius (%s deg): %s kpc' %(R_gev,dmax)

#double check my dist calc in astropy
#assume dmax, find sep btwn G150 center crds and something at the edge of the fit radius
#can I get back 60pc?
g150 = SkyCoord(l=150.3*u.degree, b=4.5*u.degree, frame='galactic',distance = dmax*u.kpc)
g150_1_4deg = SkyCoord(l=150.3*u.degree, b=5.9*u.degree, frame='galactic',distance = dmax*u.kpc)
asep = g150.separation(g150_1_4deg).deg
psep = 2*(g150.separation_3d(g150_1_4deg).to(u.pc))

print 'Assuming d=%s kpc, and sources seperated by %s deg, diam =  %s' %(dmax,asep,psep)

Using 1 GeV fit radius and d = 1.3 kpc, size = 63.5299847726
Max distance assuming 60pc diameter and > 1 GeV fit radius (1.4 deg): 1.22776670385 kpc
Assuming d=1.22776670385 kpc, and sources seperated by 1.4 deg, diam =  59.9985073858 pc


## Use 1.23 as max distance assuming SNR sizes cutoff at 60pc

# Sedov-Taylor phase stuff (see section 6 Truelove & McKee 1999)
Radius of the expanding shell

\begin{equation} R_{ST} = 0.26 (n_H/cm^{-3})^{-1/5}(t_{ST}/yr)^{2/5} (E_{ST}/4\times 10^{50} erg)^{1/5} pc = 0.31(E_{51}/n_0)^{1/5}t_{y}^{2/5} pc\end{equation}

In terms of t (this is missing a factor in front)
\begin{equation}t_y = (n_0/E_{51})^{1/2}(R_{ST}/0.31)^{5/2}\end{equation} 

Velocity of expanding shell
\begin{equation}v_s = 1.23\times 10^5(E_{51}/n_0)^{1/5}t_{y}^{2/5} km s^{-1}\end{equation}

Temperature of expanding shell (don't need this right now)
\begin{equation}T_s = 2.09\times 10^{11} = (E^{51}/n_0)^{2/5}t^{-6/5} K\end{equation}

In [11]:
#make this all work with astropy units?
def Rst(t,n0=1,E=1e51):
    """Sedov-Taylor phase SNR radius as 
       function of time. Is this 
       Chevalier 1982 or Truelove & McKee 1999? 
       I think the former? See Reynolds 2008
       
       n0 = ambient density [ cm^-3]
       t = SNR age [yr]
       E = SN explosion energy erg
       """
    return 0.314*t**(2/5.)*((E/1.e51)/n0)**(1/5.)

def Tst(R,n0=1, E=1e51):
    """Sedov-Taylor phase SNR 
        age as function of radius
       
       n0 = ambient density [ cm^-3]
       R = Sedov-Taylor phase SNR radius [pc]. 
       E = SN explosion energy erg
    """
    return (n0/(E/1e51))**(1/2)*(R/(0.314))**(5/2.)


## Test Rst and Tst on data for W44 to make sure it's right

In [12]:
#This matches Uchiyama 2012 so long as the prefactor is 0.314, bit off if it's just 0.31
r_w44 = Rst(t = 1e4, n0 = 2, E = 2e51)
t_w44 = Tst(R = 12.5,n0 = 2, E = 2e51)
print 'Test W44: R = %s, T = %s' %(r_w44,t_w44)


Test W44: R = 12.5005651554, T = 9998.86977867


## Using physical sizes calucalted above, estimate age if G150 is in ST phase
### SN explosion energy = $\mathrm{10^{51}}$ erg, ambient density  = 1 cm$\mathrm{^{-3}}$. Any reason they should be different yet?

In [13]:
T = np.vectorize(Tst)
R = np.vectorize(Rst)
T_g150 = T(R = rads,n0 = 1, E=1e51)
jT_g150 = T(R = jrads,n0 = 1, E=1e51)
jpostT_g150 = T(R = jacksSizes/2.,n0 = 1, E=1e51)
maxD_g150_T = T(R = smax/2.,n0 = 1, E=1e51)
print 'For n0 = 1 cm^-3, E_0 = 10^51 erg \n\nWith physical radii in pc %s\nAges in kyr:%s\n' %(rads,T_g150/1.e3)
print "With radii calc'd from Jack's dists in pc %s\nAges in kyr: %s\n" %(jrads,jT_g150/1.e3)
print "Using radii from poster in pc %s\nAges in kyr: %s\n" %(jacksSizes/2.0,jpostT_g150/1.e3)
print "Assuming max size of 60 pc (d=1.23kpc), \nAges in kyr: %s\n" %(maxD_g150_T/1.e3)

For n0 = 1 cm^-3, E_0 = 10^51 erg 

With physical radii in pc [ 135.79107243   96.31491116    9.39786013]
Ages in kyr:[ 3889.1431202   1647.81973863     4.90058947]

With radii calc'd from Jack's dists in pc [ 134.3903524    97.73843811   14.66076572]
Ages in kyr: [ 3789.62390902  1709.38294399    14.89592627]

Using radii from poster in pc [ 144.   104.5   15.5]
Ages in kyr: [ 4503.831251    2020.53606159    17.1200538 ]

Assuming max size of 60 pc (d=1.23kpc), 
Ages in kyr: 89.223450932



## Age of ~5kyr vs. 17kyr makes a big difference!

## Assuming typical PSR velocity, about how far offset would a ballastically traveling pulsar be from the center of SNR given estimated age (mine) from above?
### Gaensler & Slane  2006: v_psr ~400-500 km/s, but some can have an extreme value of v_psr = 1000 km/s. 


In [43]:
# T = np.vectorize(Tst)
# T_g150 = T(R = rads,n0 = 1, E=1e51)
# angSize
# size = np.vectorize(physSep)
# jsizes = size(t=R_gev*2, d=jacksDists)
print 'Projected distance of J0426 from the center is %s pc' %physSep(0.84,dnear[-1])
print 'Angular sep of J0426 from a pulsar travelling 1000km/s is %s deg' %angSize(0.1,dnear[-1])

projected distance of J0426 from the center is 5.63871607642 pc
Angular sep of J0426 from a pulsar travelling 1000km/s is 0.014897008266 deg


In [29]:
#psrVs = np.array([400.,500.,1000.])*u.km/u.s
psrVs = np.array([1.e3])*u.km/u.s
psrDs = psrVs*(T_g150*u.yr)
psrDs = psrDs.to(u.pc)
angD = np.vectorize(angSize)
angDs = angD(s=np.asarray(psrDs),d=dnear)
print "For v_psr = 1000km/s"
print "Projected pulsar distances: %s" %psrDs.to(u.pc)
print "Pulsar angular separation: %s" %angDs
print "For ages %s kyr\n" %(T_g150/1e3)

psrVs = np.array([400.])*u.km/u.s
psrDs = psrVs*(T_g150*u.yr)
psrDs = psrDs.to(u.pc)
angDs = angD(s=np.asarray(psrDs),d=dnear)
print "For v_psr = 400km/s"
print "Projected pulsar distances: %s" %psrDs.to(u.pc)
print "Pulsar angular separation: %s" %angDs
print "For ages %s kyr" %(T_g150/1e3)


For v_psr = 1000km/s
Projected pulsar distances: [ 3977.47398066  1685.24529251     5.01189246] pc
Pulsar angular separation: [ 41.00758226  24.49613856   0.74662203]
For ages [ 3889.1431202   1647.81973863     4.90058947] kyr

For v_psr = 400km/s
Projected pulsar distances: [ 1590.98959227   674.098117       2.00475699] pc
Pulsar angular separation: [ 16.4030329    9.79845543   0.29864881]
For ages [ 3889.1431202   1647.81973863     4.90058947] kyr


## 3FGL J0426 is ~0.84 deg from the center of the SNR. If the PSR was travelling at 1000 km/s, It would be ~0.1 deg away from how far out J0426 is. So it's no impossible that J0426 is the pulsar associated with G150 I guess.